In [ ]:
print("Importing the libraries")
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from FruitsNeuralNetwork import FruitsNeuralNetwork
import torch.nn.functional as F

from DbConnection import DbConnection
from FruitsDataset import FruitsDataset
import argparse
import json
from torch.utils.tensorboard import SummaryWriter

class Opt:
    epochs=None
    batch_size=None
    model_name=None
    experiment_name=None
    def __init__(self):
        self.load()
    def load(self):
        json_params= open('params.json')
        params_dict = json.load(json_params)
        self.epochs = params_dict["epochs"]
        self.batch_size = params_dict["batch_size"]
        self.model_name = params_dict["model_name"]
        self.experiment_name = params_dict["experiment_name"]
        
class FruitsTraining():
    writer={}
#    def __init__(self):

    def launch(self):
        params = self.get_params()
        self.writer = SummaryWriter(f'runs/{params.experiment_name}')
        self.train(params)

    def get_params(self):
        opt = Opt();
        return opt

    def train(self, params):
        print("Definir conexión a la BD")
        connection= DbConnection()
        print("Realizar Consulta de imágenes")
        train_dataframe= connection.get_fruits(is_test=False)
        test_dataframe= connection.get_fruits(is_test=True)
        print(train_dataframe)
        labels_dataframe= connection.get_labels()
        labels_list= labels_dataframe.iloc[:, 0]
        print("Definir datasets")
        train_dataset = FruitsDataset(train_dataframe)
        test_dataset = FruitsDataset(test_dataframe)
        print("Definir Data loaders")
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params.batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params.batch_size, shuffle=True)
        print("Obtener e imprimir la forma de los datos")
        dataiter = iter(train_loader)
        images, labels = dataiter.next()
        print("images.shape")
        print(images.shape)
        print("labels.shape")
        print(labels)
        print(labels.shape)

        print("Definir modelo")
        model = FruitsNeuralNetwork()
        print("Definir optimizador")
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        print("Definir función de pérdida")
        criterion = nn.CrossEntropyLoss()

        print("Verificar si se puede entrenar con CUDA")
        if torch.cuda.is_available():
            print("Entrenar con CUDA")
            model = model.cuda()
            criterion = criterion.cuda()
        print("Entrenar según el número de épocas")
        for epoch in range(params.epochs):
            print(f"Entrenando epoca: {epoch}")
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                images, labels = data
                if torch.cuda.is_available():
                    images = images.cuda()
                    labels = labels.cuda()

                print("zero the parameter gradients")
                optimizer.zero_grad()

                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # normalized_labels = []
                # for Lbl in labels:
                #     normalized_labels.append([int(1) if x == Lbl else int(0) for x in labels_list])
                # normalized_labels = torch.tensor(np.asarray(normalized_labels), dtype=torch.long)

                #normalized_labels = torch.tensor(np.asarray([[x] for x in labels]))
                running_loss += loss.item()

                if i % 1000 == 999:    # every 1000 mini-batches...

                    # ...log the running loss
                    self.writer.add_scalar('training loss',
                                    running_loss / 1000,
                                    epoch * len(train_loader) + i)

                    # ...log a Matplotlib Figure showing the model's predictions on a
                    # random mini-batch
                    self.writer.add_figure('predictions vs. actuals',
                                    self.plot_classes_preds(model, images, labels, labels_list),
                                    global_step=epoch * len(train_loader) + i)
                    running_loss = 0.0
                print("Guardar modelo")
                self.save(model, params.model_name)
            else:
                print("Epoch {} - Training loss: {}".format(epoch+1, running_loss/len(train_loader)))
        print("Realizar prueba de modelo")
        self.test(test_loader=test_loader, model=model)

    def save(self, model, path):
        torch.save(model, path)

    def test(self, test_loader, model):
        print("Getting predictions on test set and measuring the performance")
        correct_count, all_count = 0, 0
        for images,labels in test_loader:
            for i in range(len(labels)):
                if torch.cuda.is_available():
                    images = images.cuda()
                    labels = labels.cuda()
                img = images[i].view(1, 1, 28, 28)
                with torch.no_grad():
                    logps = model(img)
                ps = torch.exp(logps)
                probab = list(ps.cpu()[0])
                pred_label = probab.index(max(probab))
                true_label = labels.cpu()[i]
                if(true_label == pred_label):
                    correct_count += 1
                all_count += 1

        print("Number Of Images Tested =", all_count)
        print("\nModel Accuracy =", (correct_count/all_count))

    def images_to_probs(self, net, images):
        '''
        Generates predictions and corresponding probabilities from a trained
        network and a list of images
        '''
        output = net(images)
        # convert output probabilities to predicted class
        _, preds_tensor = torch.max(output, 1)
        preds = np.squeeze(preds_tensor.numpy())
        return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]

    def plot_classes_preds(self, net, images, labels, classes):
        '''
        Generates matplotlib Figure using a trained network, along with images
        and labels from a batch, that shows the network's top prediction along
        with its probability, alongside the actual label, coloring this
        information based on whether the prediction was correct or not.
        Uses the "images_to_probs" function.
        '''
        preds, probs = self.images_to_probs(net, images)
        # plot the images in the batch, along with predicted and true labels
        fig = plt.figure(figsize=(12, 48))
        for idx in np.arange(4):
            ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
            self.matplotlib_imshow(images[idx], one_channel=True)
            ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
                classes[preds[idx]],
                probs[idx] * 100.0,
                classes[labels[idx]]),
                        color=("green" if preds[idx]==labels[idx].item() else "red"))
        return fig

    def matplotlib_imshow(self, img, one_channel=False):
        if one_channel:
            img = img.mean(dim=0)
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        if one_channel:
            plt.imshow(npimg, cmap="Greys")
        else:
            plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
%tensorboard --logdir runs

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances
notebook.display(port=6006, height=1000) 
FruitsTraining().launch()